## Homework 09: The Return of The Ten Arcs
##### By: Kevin Liu

In [1]:
import numpy as np
import pandas as pd
from scipy.special import logsumexp
from collections import Counter
import re
import string

### 1. Write a Simulator as a Positive Control

In this problem set, we are interested in assessing Lestrade's claim that both Arc2 and Arc3 are strongly on when it was previously thought that Arc2 and Arc3 are oppositely regulated.

Before we begin our analysis, we will first utilize a generative probability model based on the works of Li and Dewey (2010) to generate positive control data in order to assess the performance of Lestrade's method. Specifically, we will simulate N = 1000000 observed read counts based on a simulated Arc locus structure with lengths $L_i$ and transcript abundances $\tau$ that follows the problem set's specified model.

In [2]:
def mk_tau_len(n_transcripts, min_len = 2, max_len = 4):
    """
    Generates random transcript abundances and lengths between mix_len and max_len.
    """
    tau_arr = np.random.dirichlet(np.ones(n_transcripts), size = 1)[0]
    len_arr = np.random.randint(min_len, max_len+1, n_transcripts)
    
    return tau_arr, len_arr


def mk_transcripts(len_arr):
    """
    Generates a number of transcripts (i.e., Arc loci) based on a supplied list of transcript lengths 
    as a list of lists, where each nested list element represents a read.
    """
    transcript_lst = []
    n_segments = len(len_arr)
    segment_lst = [i for i in range(n_segments)]
    
    for segment_start_pos, len_transcript in enumerate(len_arr):
        segment_end_pos = segment_start_pos+len_transcript
        if segment_end_pos > n_segments:
            transcript = segment_lst[segment_start_pos:]+segment_lst[:segment_end_pos-n_segments]
            transcript_lst.append(list(transcript))
        else: 
            transcript_lst.append(segment_lst[segment_start_pos:segment_end_pos])
    
    return transcript_lst


def tau_to_nu(tau_arr, len_arr):
    """
    Converts transcript abundances to nucleotide abundances.
    """
    nu_arr = tau_arr*len_arr
    nu_arr = nu_arr/sum(nu_arr)
    
    return nu_arr


def nu_to_tau(nu_arr, len_arr):
    """
    Converts nucleotide abundances to transcript abundances.
    """    
    tau_arr = nu_arr/len_arr
    tau_arr = tau_arr/sum(tau_arr)
    
    return tau_arr


def mk_reads(transcript_lst, tau_arr, len_arr, n_reads = 1000000):
    """
    Samples n_reads number of reads from all possible transcripts based on the nucleotide abundances 
    and transcript lengths. 
    """
    n_transcripts = len(transcript_lst)
    nu_arr = tau_to_nu(tau_arr, len_arr)
    read_lst = []
    for i in range(n_reads):
        transcript_idx = np.random.choice(n_transcripts, p = nu_arr)
        transcript = transcript_lst[transcript_idx]
        segment = np.random.choice(transcript)
        read_lst.append(segment)
        
    return read_lst

In [3]:
tau_arr, len_arr = mk_tau_len(n_transcripts = 10, min_len = 2, max_len = 4)
transcript_lst = mk_transcripts(len_arr)
read_lst = mk_reads(transcript_lst, tau_arr, len_arr, n_reads = 1000000)

In [4]:
pd.DataFrame({'Length': len_arr, 
              'Sequence': [[j+1 for j in i] for i in transcript_lst], 
              '$\tau_i$': tau_arr})

,Length,Sequence,$\tau_i$
0,2,"[1, 2]",0.099692
1,3,"[2, 3, 4]",0.128561
2,4,"[3, 4, 5, 6]",0.005822
3,3,"[4, 5, 6]",0.049498
4,4,"[5, 6, 7, 8]",0.216202
5,4,"[6, 7, 8, 9]",0.010340
6,2,"[7, 8]",0.051502
7,2,"[8, 9]",0.311506
8,3,"[9, 10, 1]",0.125515
9,3,"[10, 1, 2]",0.001362


In [5]:
df = pd.DataFrame.from_dict(Counter(read_lst), orient = "index").reset_index()
df.columns = ["Read_Sequence", "Counts"]
df.Read_Sequence = df.Read_Sequence+1
sorted_df = df.sort_values("Read_Sequence")
sorted_df

,Read_Sequence,Counts
9,1,81775
3,2,82848
0,3,48485
4,4,66701
2,5,97708
8,6,101595
6,7,100631
1,8,212843
7,9,161485
5,10,45929


### 2. Calculate the Log Likelihood

Since we have generated simulated positive control data with known transcript abudnances $\tau$ and lengths $L_i$, we will begin to assess Lestrade's method in estimating the parameters by comparing it to our known true parameters through calculating the log-likelihoods. To implement this for a given Arc locus structure, we calculate the log probability of the observed read counts $r_k$ given that the model and parameters $\tau$ and $L$ are known.

In [6]:
def nll(read_lst, transcript_lst, tau_arr, len_arr):
    """
    Calculates the negative log-likelihood of transcript abundances.
    """
    nu_arr = tau_to_nu(tau_arr, len_arr)
    log_prob_lst = []
    unique_read_arr, unique_read_count_arr = np.unique(read_lst, return_counts = True)
    
    for read_idx, read in enumerate(unique_read_arr):
        read_prob_lst = []
        for transcript_idx, transcript in enumerate(transcript_lst):
            if read in transcript:
                read_prob_lst.append(np.log(nu_arr[transcript_idx]/len_arr[transcript_idx]))
        
        log_prob_lst.append(logsumexp(read_prob_lst)*unique_read_count_arr[read_idx])
        
    nll = -np.sum(log_prob_lst)
    
    return nll


def get_les_tau(r, T, L):
    """
    Adapted from w09-naive.py.
    Generates transcript abundance estimates using Lestrade's method.
    """
    S = T    # S = R = T : there are T transcripts (Arc1..Arc10), S segments (A..J), R reads (a..j)

    # Count how often each segment A..J is used in the isoforms i
    # We'll use that to split observed read counts across the isoforms
    # that they might have come from.
    #
    segusage = np.zeros(S).astype(int)
    for i in range(T):
        for j in range(i,i+L[i]): 
            segusage[j%S] += 1

    # Naive analysis:
    #
    c  = np.zeros(T)
    for i in range(T):
        for k in range(i,i+L[i]):
            c[i] += (1.0 / float(segusage[k%S])) * float(r[k%S])  # For each read k, assume read k-> segment j,
                                                                  # and assign 1/usage count to each transcript
                                                                  # that contains segment j.
    Z       = np.sum(c)
    est_nu  = np.divide(c, Z)       # nucleotide abundance
    est_tau = np.divide(est_nu, L)  # convert to TPM, transcript abundance
    est_tau = np.divide(est_tau, np.sum(est_tau))
    
    return est_tau

In [7]:
les_tau_arr = get_les_tau(sorted_df["Counts"], len(transcript_lst), len_arr)
true_nll = nll(read_lst, transcript_lst, tau_arr, len_arr)
les_nll = nll(read_lst, transcript_lst, les_tau_arr, len_arr)

In [8]:
pd.DataFrame({"True $\tau_i$": tau_arr,
              "Lestrade's $\hat\tau_i$": les_tau_arr,
              "$\Delta\tau_i$": tau_arr-les_tau_arr})

,True $\tau_i$,Lestrade's $\hat\tau_i$,$\Delta\tau_i$
0,0.099692,0.126446,-0.026754
1,0.128561,0.142659,-0.014098
2,0.005822,0.079972,-0.074150
3,0.049498,0.059352,-0.009853
4,0.216202,0.078117,0.138085
5,0.010340,0.086559,-0.076219
6,0.051502,0.107293,-0.055791
7,0.311506,0.107759,0.203747
8,0.125515,0.087979,0.037536
9,0.001362,0.123865,-0.122503


In [9]:
print("Negative log-likelihood of true parameters: " + str(np.round(true_nll, 3)))
print("Negative log-likelihood of Lestrade's parameters: " + str(np.round(les_nll, 3)))
print("Difference between negative log-likelihoods: " + str(np.abs(np.round(true_nll-les_nll, 3))))

Negative log-likelihood of true parameters: 2194330.836
Negative log-likelihood of Lestrade's parameters: 2271788.715
Difference between negative log-likelihoods: 77457.879


Based on the above table comparing the true transcript abundances to Lestrade's transcript abundances, it is apparent that Lestrade's method of estimating transcript abundances are different from our positive control true transcript abudnances. This difference is reinforced by the calculated difference in negative log-likelihoods, where the negative log-likelihood of the true parameters is significantly larger than that of Lestrade's parameters, suggesting that Lestrade's estimates are significantly less likely than the true parameters.

We will attempt to optimize the negative log-likelihoods by using an expectation maximization (EM) algorithm to obtain better parameter estimates.

### 3. Estimate Isoform Abundances by EM

Here, we will write an EM algorithm to estimate the unknown isoform abundances $\tau_i$ for each isoform Arc1, ..., Arc10, given read counts $r_k$ and the structure of the Arc locus including the lengths $L_i$.

In [10]:
def read_data(filename):
    """
    Adapted from w09-naive.py.
    Reads in a data table and returns the read counts and transcript lengths.
    """
    with open(filename) as f:
        #   The first line is "The transcripts of the sand mouse Arc locus"
        line  = f.readline()
        match = re.search(r'^The (\d+) transcripts', line)
        T     = int(match.group(1))

        # The next T lines are 
        #       
        # tau's may be present, or obscured ("xxxxx")
        tau       = np.zeros(T)
        L         = np.zeros(T).astype(int)
        tau_known = True   # until we see otherwise
        for i in range(T):
            fields    = f.readline().split()
            if fields[1] == "xxxxx":
                tau_known = False
            else:
                tau[i] = float(fields[1])
            L[i]      = int(fields[2])

        # after a blank line,
        # 'The  read sequences':
        line  = f.readline()
        line  = f.readline()
        match = re.search(r'The (\d+) read sequences', line)
        N     = int(match.group(1))

        # the next T lines are 
        #
        r = np.zeros(T).astype(int)
        for k in range(T):
            fields = f.readline().split()
            r[k]   = fields[1]
    
    return r, L
    

def EM(read_lst, transcript_lst, len_arr, threshold = 0.001):
    """
    Perform EM optimization with a maximum of n_iterations given parameters.
    """
    tau_arr = mk_tau_len(n_transcripts = len(len_arr), min_len = 2, max_len = 4)[0]
    nu_arr = nu_to_tau(tau_arr, len_arr)
    unique_read_arr, unique_read_count_arr = np.unique(read_lst, return_counts = True)

    nll_new = 0
    nll_old = nll(read_lst, transcript_lst, tau_arr, len_arr)
    nll_lst = [nll_old]

    while np.abs(nll_new - nll_old) > threshold:
        nll_old = nll(read_lst, transcript_lst, tau_arr, len_arr)
        count_arr = np.zeros(10)

        for read_idx, read in enumerate(unique_read_arr):
            read_prob_lst = []
            read_idx_lst = []
            for transcript_idx, transcript in enumerate(transcript_lst):
                if read in transcript:
                    read_idx_lst.append(transcript_idx)
                    num = np.log(nu_arr[transcript_idx]/len_arr[transcript_idx])
                    read_prob_lst.append(num)

            read_prob_lst -= logsumexp(read_prob_lst)
            np.add.at(count_arr, read_idx_lst, np.exp(read_prob_lst)*unique_read_count_arr[read_idx])

        nu_arr = count_arr/np.sum(count_arr)
        tau_arr = nu_to_tau(nu_arr, len_arr)
        nll_new = nll(read_lst, transcript_lst, tau_arr, len_arr)
        nll_lst.append(nll_new)

    print("Number of iterations to convergence: " + str(len(nll_lst)-1))
    
    return tau_arr, nll_lst

We will first run our EM algorithm, calculate the negative log-likelihood of the estimated parameters, and compare it to that of the true parameters from our simulated positive control data to assess our EM algorithm performance.

In [11]:
em_tau_arr, em_nll_lst = EM(read_lst, transcript_lst, len_arr)

Number of iterations to convergence: 3553


In [12]:
pd.DataFrame({"EM $\hat\tau_i$": em_tau_arr,
              "True $\hat\tau_i$": tau_arr,
              "$\Delta\hat\tau_i$": em_tau_arr-tau_arr})

,EM $\hat\tau_i$,True $\hat\tau_i$,$\Delta\hat\tau_i$
0,0.099160,0.099692,-0.000532
1,0.124728,0.128561,-0.003832
2,0.009391,0.005822,0.003569
3,0.050273,0.049498,0.000774
4,0.209904,0.216202,-0.006299
5,0.011788,0.010340,0.001448
6,0.056003,0.051502,0.004502
7,0.311683,0.311506,0.000176
8,0.122136,0.125515,-0.003380
9,0.004936,0.001362,0.003574


In [13]:
print("Negative log-likelihood of true parameters: " + str(np.round(true_nll, 3)))
print("Negative log-likelihood of EM parameters: " + str(np.round(em_nll_lst[-1], 3)))
print("Difference between negative log-likelihoods: " + str(np.abs(np.round(true_nll-em_nll_lst[-1], 3))))

Negative log-likelihood of true parameters: 2194330.836
Negative log-likelihood of EM parameters: 2194330.094
Difference between negative log-likelihoods: 0.742


Based on the results shown above, we observe that there is a marginal difference in the negative log-likelihoods between the true parameters and the EM parameters, suggesting that our EM algorithm is both accurate and robust in estimating the unknown isoform abundances $\tau_i$ using the simulated positive control data.

Subsequently, we will read in the supplementary data from Lestrade et al., compare our EM algorithm's performance to that of Lestrade's method, and examine Lestrade's claim that Arc2 and Arc3 are both strongly on.

In [14]:
r, L = read_data("w09-data.out")
data_read_lst = []

for read_idx, read_count in enumerate(r):
    data_read_lst += [read_idx]*read_count
    
data_transcript_lst = mk_transcripts(L)

data_em_tau_arr, data_em_nll_lst = EM(data_read_lst, data_transcript_lst, L)

Number of iterations to convergence: 1544


In [15]:
data_les_tau_arr = get_les_tau(r, len(data_transcript_lst), L)
data_em_nll = nll(data_read_lst, data_transcript_lst, data_em_tau_arr, L)
data_les_nll = nll(data_read_lst, data_transcript_lst, data_les_tau_arr, L)

In [16]:
data_em_df = pd.DataFrame({"Isoform": ["Arc" + str(i+1) for i in range(len(L))],
                           "Lestrade's $\hat\tau_i$": data_les_tau_arr,
                           "EM $\hat\tau_i$": data_em_tau_arr,
                           "$\Delta\hat\tau_i$": data_les_tau_arr-data_em_tau_arr})
data_em_df

,Isoform,Lestrade's $\hat\tau_i$,EM $\hat\tau_i$,$\Delta\hat\tau_i$
0,Arc1,0.165987,0.147305,0.018683
1,Arc2,0.248437,0.497552,-0.249115
2,Arc3,0.115051,0.002057,0.112993
3,Arc4,0.044097,0.049636,-0.005539
4,Arc5,0.039440,0.036410,0.003030
5,Arc6,0.041721,0.019620,0.022101
6,Arc7,0.043045,0.006656,0.036389
7,Arc8,0.079994,0.135416,-0.055422
8,Arc9,0.085655,0.053807,0.031848
9,Arc10,0.136573,0.051542,0.085031


In [17]:
print("Negative log-likelihood of EM parameters: " + str(np.round(data_em_nll, 3)))
print("Negative log-likelihood of Lestrade's parameters: " + str(np.round(data_les_nll, 3)))
print("Difference between negative log-likelihoods: " + str(np.abs(np.round(data_em_nll-data_les_nll, 3))))

Negative log-likelihood of EM parameters: 2021934.129
Negative log-likelihood of Lestrade's parameters: 2084370.855
Difference between negative log-likelihoods: 62436.726


As shown from the above results, our EM algorithm is superior to that of Lestrade's approach given the large difference in the negative log-likelihoods, where our EM estimated parameters yielded negative log-likelihoods that are much larger than that of Lestrade's estimated parameters. Furthermore, we see from the $\Delta\hat\tau_i$ in the table of estimated transcript abundances that Lestrade's method underestimates Arc2 and overestimates Arc3 relative to our EM algorithm, which might explain his unlikely claim that Arc2 and Arc3 are both strongly on.

Next, we will further examine Lestrade's claim by extracting the top two isoforms that have the largest and smallest transcript abundances.

In [18]:
sorted_data_em_df = data_em_df.sort_values("EM $\hat\tau_i$", ascending = False)

max2_tau_df = sorted_data_em_df.head(2)
min2_tau_df = sorted_data_em_df.tail(2)

print("The two most abundant transcripts are " + 
      str(' and '.join([str(i) for i in list(max2_tau_df["Isoform"])])) + 
      ", which accounts for " + str(round(max2_tau_df["EM $\hat\tau_i$"].sum()*100, 1)) + 
      "% of the population together.")

print("The two least abundant transcripts are " + 
      str(' and '.join([str(i) for i in list(min2_tau_df["Isoform"])])) + 
      ", which accounts for " + str(round(min2_tau_df["EM $\hat\tau_i$"].sum()*100, 3)) + 
      "% of the population together.")

The two most abundant transcripts are Arc2 and Arc1, which accounts for 64.5% of the population together.
The two least abundant transcripts are Arc7 and Arc3, which accounts for 0.871% of the population together.


With our previous findings in mind, we further observe that Arc2 is one of the most abundant transcripts and that Arc3 is one of the least abundant transcripts, based on such evidence we can conclude that Lestrade's claim that Arc2 and Arc3 are both strongly on is false.

Lestrade's method is expectedly poor performing as we notice that his method uniformly assigns read counts to the respective isoforms in an arbitrary manner, which is analogous to running a single iteration of our EM algorithm with the initial parameters. Therefore, it is unsurprising that our iterative EM optimization algorithm outperforms Lesetrade's method.